In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "5"
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import time
from utils.config import load_configs
from data.preprocessing import build_feature_columns
from models.esmm import ESMM
from data.dataset import NewsDataset
import tensorflow as tf
tf.get_logger().setLevel("ERROR")
import datetime
from tensorflow.keras.callbacks import TensorBoard
import numpy as np
import polars as pl

In [2]:
loaded_model = tf.keras.models.load_model('/data3/zxh/news_rec/temp_results/essm/003/')

2025-04-06 01:24:38.230881: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-04-06 01:24:41.130196: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1510] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 22423 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3090, pci bus id: 0000:89:00.0, compute capability: 8.6
2025-04-06 01:24:45.278823: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2025-04-06 01:24:47.506377: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 230400288 exceeds 10% of free system memory.
2025-04-06 01:24:54.851161: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocati

In [3]:
loaded_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
age (InputLayer)                [(None, None)]       0                                            
__________________________________________________________________________________________________
gender (InputLayer)             [(None, None)]       0                                            
__________________________________________________________________________________________________
keywords (InputLayer)           [(None, None)]       0                                            
__________________________________________________________________________________________________
os (InputLayer)                 [(None, 1)]          0                                            
______________________________________________________________________________________________

In [3]:
# 加载配置
config = load_configs()

# 加载特征列
feature_columns, feature_groups = build_feature_columns(config['feature'])

# 初始化数据集处理器
dataset_creator = NewsDataset(config, feature_columns)

# 创建测试数据集
dataset = {
    "test" : dataset_creator.create_dataset(data_path=config['training']['global']['test_data']),
}

test_dataset = dataset["test"].map(lambda x, y: x)  # 只取输入特征

In [4]:
# 读取df版本的测试数据
test_data = pl.read_csv("/data3/zxh/news_rec/rank_csv_data/test_data/test_data.csv",separator="\t")
test_data

user_id,article_id,expose_time,network_env,refresh_count,expose_pos,is_clicked,duration,device_name,os,province,city,age,gender,userid_history_duration_mean,userid_history_count,userid_expose_count,userid_ctr,keywords,image_count,category_level1,category_level2,docid_history_count,docid_expose_count,docid_ctr,docid_history_duration_mean,category1_ctr,category1_popularity,category1_history_duration_mean,category2_ctr,category2_popularity,category2_history_duration_mean,userid_category1_history_count,userid_category1_ctr,userid_category1_history_duration_mean,userid_category2_history_count,userid_category2_ctr,userid_category2_history_duration_mean,expose_hourdiff,expose_hour,is_converted
i64,i64,i64,i64,f64,i64,i64,f64,str,str,str,str,str,str,f64,f64,f64,f64,str,i64,str,str,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,f64,f64,i64,f64,f64,i64,i64,i64
2431381002,466497559,1625560435365,2,0.366121,0,0,0.0,"""M2007J22C""","""Android""","""江西""","""九江""","""A_0_24:0.084074,A_25_29:0.0495…","""female:0.276102,male:0.723898""",0.316542,0.844135,0.76849,0.127043,"""人生:0.035545,劈腿:0.036345,包间:0.0…",3,"""情感""","""情感/婚姻与家庭""",0.319737,0.353534,0.006392,0.283393,0.157579,0.108771,0.363147,0.15524,0.082245,0.378103,3,0.247863,0.309052,3,0.217391,0.34117,16,8,0
2390152616,466838383,1625584316582,2,0.829975,0,0,0.0,"""V1965A""","""Android""","""辽宁""","""葫芦岛""","""A_0_24:0.066269,A_25_29:0.0717…","""female:0.273217,male:0.726783""",0.241778,0.896009,0.866664,0.077854,"""4s店:0.023849,事故:0.029743,二手车:0…",4,"""汽车""","""汽车/用车""",0.319737,0.353534,0.006392,0.283393,0.117285,0.018259,0.307542,0.114759,0.005196,0.326605,3,0.097902,0.266277,2,0.12,0.317765,4,15,0
2443013308,466596360,1625552213014,2,0.138112,0,0,0.0,"""LIO-AN00""","""Android""","""广东""","""东莞""","""A_0_24:0.09405,A_25_29:0.17353…","""female:0.629811,male:0.370189""",0.25391,0.894818,0.769411,0.282008,"""低胸:0.032807,低胸^^黑裙:0.026965,发型…",4,"""时尚""","""时尚/女性时尚""",0.319737,0.353534,0.006392,0.283393,0.14042,0.028536,0.194861,0.118006,0.010281,0.233038,3,0.424,0.210312,3,0.4,0.228574,4,6,0
1293444900,466550480,1625547205088,5,0.21485,0,0,0.0,"""MHA-AL00""","""Android""","""海南""","""三亚""","""A_0_24:0.020534,A_25_29:0.1824…","""female:0.0,male:1.0""",0.238909,0.906673,0.802173,0.231358,"""naval:0.056106,naval^^news:0.0…",4,"""军事""","""军事/武器""",0.319737,0.353534,0.006392,0.283393,0.16552,0.056195,0.347863,0.160136,0.011546,0.345481,3,0.310345,0.222759,3,0.351351,0.228955,6,4,0
2446511758,465769818,1625546678713,2,0.26756,0,1,0.57374,"""""","""""","""""","""""","""A_0_24:0.282267,A_25_29:0.2958…","""female:0.561326,male:0.438674""",0.487107,0.773608,0.64507,0.200875,"""亲情:0.018269,储君:0.027646,刘应:0.0…",4,"""历史""","""历史/中国史""",0.886911,0.816734,0.052409,0.396306,0.128401,0.036799,0.393342,0.124567,0.030694,0.399198,3,0.35,0.521977,3,0.352941,0.52245,69,4,1
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
2445496462,466025645,1625530203987,2,0.21485,0,0,0.0,"""Mi9Pro5G""","""Android""","""广东""","""广东""","""A_0_24:0.203706,A_25_29:0.2327…","""female:0.0,male:1.0""",0.347217,0.331481,0.348857,0.036223,"""人力^^资源:0.023107,人力^^资源委员会:0.02…",3,"""财经""","""财经/财经人物""",0.830543,0.707137,0.099644,0.342895,0.129982,0.024365,0.335007,0.140624,0.004679,0.354351,null,0.25,0.34075,null,0.333333,0.339603,41,0,0
1349872856,466714829,1625567748252,2,0.64565,0,0,0.0,"""PACM00""","""Android""","""山东""","""济宁""","""A_0_24:0.088099,A_25_29:0.5715…","""female:0.0,male:1.0""",0.355585,0.44868,0.699602,0.005433,"""乳汁:0.028392,二手烟:0.023179,产后:0.…",4,"""育儿""","""育儿/孕产健康护理""",0.319737,0.353534,0.006392,0.283393,0.128413,0.01929,0.319762,0.128096,0.007774,0.328575,1,0.066667,0.327847,null,0.333333,0.339603,8,10,0
2445791558,466446910,1625540321832,5,0.138112,0,0,0.0,"""V1831A""","""Android""","""浙江""","""杭州""","""A_0_24:0.508554,A_25_29:0.2355…","""female:0.252032,male:0.747968""",0.347217,0.0,0.784973,-4.3152e-19,"""低胸:0.041147,健美:0.028697,克洛伊:0.…",4,"""娱乐""","""娱乐/欧美明星""",0.319737,0.353534,0.

In [ ]:
ctr_pred,cvr_pred = loaded_model.predict(test_dataset)
test_data = test_data.select(["user_id","article_id","expose_time","duration","is_clicked"]).with_columns(pl.Series("ctr_pred", ctr_pred.flatten()))

2025-04-04 15:36:44.770597: I tensorflow/stream_executor/cuda/cuda_blas.cc:1760] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


ColumnNotFoundError: ctr_pred

Resolved plan until failure:

	---> FAILED HERE RESOLVING 'sink' <---
DF ["user_id", "article_id", "expose_time", "network_env", ...]; PROJECT */41 COLUMNS

In [ ]:
test_data = test_data.sort(["ctr_pred"],descending=True)
test_data

user_id,article_id,expose_time,duration,is_clicked,ctr_pred
i64,i64,i64,f64,i64,f32
2445841240,466709026,1625573607601,0.18353,1,0.969674
2445841240,466741892,1625573607601,0.167981,1,0.961923
1365852424,466489688,1625540462922,0.369892,1,0.958915
1669355924,466497595,1625542758264,0.127238,1,0.951386
2384535280,466495809,1625554970395,0.113569,1,0.949852
…,…,…,…,…,…
1386651770,466200120,1625550810754,0.0,0,0.00005
1999962374,466610267,1625574259494,0.0,0,0.000042
2446977424,466200120,1625540286797,0.0,0,0.000041


In [9]:
# 以 user_id 和 expose_time 进行分组，并统计每个分组的数量
grouped_data = test_data.group_by(["user_id", "expose_time"]).agg(pl.len().alias("count"))

# 过滤出 count >= 5 的数据
filtered_data = grouped_data.filter(pl.col("count") >= 5)

# 以 user_id 和 expose_time 进行匹配，筛选出 test_data 中符合 filtered_data 的数据
matched_data = test_data.join(filtered_data, on=["user_id", "expose_time"], how="inner").drop(["count"])

# 输出结果
matched_data = matched_data.sort(["user_id","expose_time"])

matched_data

user_id,article_id,expose_time,duration,is_clicked,ctr_pred
i64,i64,i64,f64,i64,f32
17340,466126343,1625571013757,0.368962,1,0.445067
17340,466368404,1625571013757,0.172085,1,0.471752
17340,466699030,1625571013757,0.0,0,0.430687
17340,466090289,1625571013757,0.0,0,0.370488
17340,466399808,1625571013757,0.0,0,0.395077
…,…,…,…,…,…
2447273764,466826642,1625587022525,0.0,0,0.058803
2447273764,466823351,1625587022525,0.0,0,0.112369
2447273764,466036199,1625587022525,0.0,0,0.083002


In [25]:
# 提取真实标签和预测概率
y_true = test_data["is_clicked"].to_numpy()
y_pred_prob = test_data["ctr_pred"].to_numpy()

# 计算 AUC
auc = roc_auc_score(y_true, y_pred_prob)

# 计算二分类预测值 (阈值设为 0.5)
y_pred = (y_pred_prob >= 0.5).astype(int)

# 计算 F1-score
f1 = f1_score(y_true, y_pred)

# 计算二分类准确率
accuracy = accuracy_score(y_true, y_pred)

# 输出结果
print(f"AUC: {auc:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"Accuracy: {accuracy:.4f}")

AUC: 0.7527
F1-score: 0.1907
Accuracy: 0.8610


In [27]:
# 存储每组的 AUC 和样本数
group_aucs = []
group_sizes = []

# 按 (user_id, expose_time) 分组
for (uid, time), group in test_data.group_by(['user_id', 'expose_time']):
    if group['is_clicked'].n_unique() < 2:
        continue  # AUC 不定义（全是 0 或全是 1）
    auc = roc_auc_score(group['is_clicked'], group['ctr_pred'])
    group_aucs.append(auc * len(group))
    group_sizes.append(len(group))

# 计算加权平均 GAUC
gauc = sum(group_aucs) / sum(group_sizes)
print(f"GAUC: {gauc:.6f}")

GAUC: 0.612186
